# Ejercicio 9: Uso de la API de OpenAI

En este ejercicio vamos a aprender a utilizar la API de OpenAI

## 1. Uso básico

El siguiente código sirve para conectarse con la API de OpenAI de forma básica

In [1]:
from openai import OpenAI

In [4]:
OPENAI_API_KEY = ''

In [6]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [7]:
response = client.responses.create(
    model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

Under a silver moon, a gentle unicorn named Luna tiptoed through a field of starlight flowers, sprinkling sweet dreams wherever her sparkling hooves touched the ground.


## 2. Retrieval

### 2.1 Cargo el corpus de 20 News Groups

In [8]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroupsdocs = newsgroups.data

In [9]:
import pandas as pd

In [22]:
newsgroups_df = pd.DataFrame(newsgroupsdocs, columns=['raw'])
newsgroups_df

,raw
0,"\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game. PENS RULE!!!\n\n"
1,My brother is in the market for a high-performance video card that supports\nVESA local bus with 1-2MB RAM. Does anyone have suggestions/ideas on:\n\n - Diamond Stealth Pro Local Bus\n\n - Orchid Farenheit 1280\n\n - ATI Graphics Ultra Pro\n\n - Any other high-performance VLB card\n\n\nPlease post or email. Thank you!\n\n - Matt\n
2,"\n\n\n\n\tFinally you said what you dream about. Mediterranean???? That was new....\n\tThe area will be ""greater"" after some years, like your ""holocaust"" numbers......\n\n\n\n\n\t\t*****\n\tIs't July in USA now????? Here in Sweden it's April and still cold.\n\tOr have you changed your calendar???\n\n\n\t\t\t\t\t\t ****************\n\t\t\t\t\t\t ******************\n\t\t\t ***************\n\n\n\tNOTHING OF THE MENTIONED IS TRUE, BUT LET SAY IT's TRUE.\n\t\n\tSHALL THE AZERI WOMEN AND CHILDREN GOING TO PAY THE PRICE WITH\n\t\t\t\t\t\t **************\n\tBEING RAPED, KILLED AND TORTURED BY THE ARMENIANS??????????\n\t\n\tHAVE YOU HEARDED SOMETHING CALLED: ""GENEVA CONVENTION""???????\n\tYOU FACIST!!!!!\n\n\n\n\tOhhh i forgot, this is how Armenians fight, nobody has forgot\n\tyou killings, rapings and torture against the Kurds and Turks once\n\tupon a time!\n \n \n\n\nOhhhh so swedish RedCross workers do lie they too? What ever you say\n""regional killer"", if you don't like the person then shoot him that's your policy.....l\n\n\n\t\t\t\t\t\t\t\t\t\ti\n\t\t\t\t\t\t\t\t\t\ti\n\t\t\t\t\t\t\t\t\t\ti\n\tConfused?????\t\t\t\t\t\t\t\ti\n\t\t\t\t\t\t\t\t\t\ti\n Search Turkish planes? You don't know what you are talking about.\ti\n Turkey's government has announced that it's giving weapons <-----------i\n to Azerbadjan since Armenia started to attack Azerbadjan\t\t\n it self, not the Karabag province. So why search a plane for weapons\t\n since it's content is announced to be weapons? \n\n\tIf there is one that's confused then that's you! We have the right (and we do)\n\tto give weapons to the Azeris, since Armenians started the fight in Azerbadjan!\n \n\n\n\tShoot down with what? Armenian bread and butter? Or the arms and personel \n\tof the Russian army?\n\n\n"
3,\nThink!\n\nIt's the SCSI card doing the DMA transfers NOT the disks...\n\nThe SCSI card can do DMA transfers containing data from any of the SCSI devices\nit is attached when it wants to.\n\nAn important feature of SCSI is the ability to detach a device. This frees the\nSCSI bus for other devices. This is typically used in a multi-tasking OS to\nstart transfers on several devices. While each device is seeking the data the\nbus is free for other commands and data transfers. When the devices are\nready to transfer the data they can aquire the bus and send the data.\n\nOn an IDE bus when you start a transfer the bus is busy until the disk has seeked\nthe data and transfered it. This is typically a 10-20ms second lock out for other\nprocesses wanting the bus irrespective of transfer time.\n
4,"1) I have an old Jasmine drive which I cannot use with my new system.\n My understanding is that I have to upsate the driver with a more modern\none in order to gain compatability with system 7.0.1. does anyone know\nof an inexpensive program to do this? ( I have seen formatters for <$20\nbuit have no idea if they will work)\n \n2) I have another an

### 2.2 Transformo a embeddings

In [12]:
from sentence_transformers import SentenceTransformer
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [16]:
model = SentenceTransformer('all-MiniLM-L6-v2')

C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
embeddings = model.encode(newsgroupsdocs)

In [18]:
embeddings

array([[ 0.00207798,  0.02345042,  0.02480885, ...,  0.0014359 ,
         0.0151075 ,  0.0528758 ],
       [ 0.0500603 ,  0.02698093, -0.00886483, ..., -0.0088717 ,
        -0.06737082,  0.05656359],
       [ 0.01640468,  0.08100048, -0.04953597, ..., -0.04184627,
        -0.07800218, -0.0313095 ],
       ...,
       [-0.00509084,  0.01817272,  0.04388077, ...,  0.01331368,
        -0.05997062, -0.05430663],
       [ 0.03508158, -0.05842971, -0.03385155, ..., -0.02824297,
        -0.05223113,  0.03760364],
       [-0.06498063, -0.01133726,  0.03949648, ..., -0.03573753,
         0.07217911,  0.02192114]], dtype=float32)

In [25]:
newsgroups_df['embeddings'] = embeddings.tolist()
newsgroups_df

,raw,embeddings
0,"\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game. PENS RULE!!!\n\n","[0.0020779806654900312, 0.023450423032045364, 0.0248088538646698, -0.010110200382769108, 0.04626138508319855, -0.019038811326026917, 0.06198830157518387, 0.049166589975357056, 0.026586145162582397, -0.009346400387585163, -0.0995098128914833, 0.039723265916109085, -0.05520962178707123, 0.02532419189810753, 0.02993602119386196, -0.019566623494029045, -0.06086605787277222, 0.015870140865445137, 0.025333929806947708, 0.04593871906399727, -0.014141423627734184, -0.007948887534439564, 0.021375149488449097, -0.010109618306159973, 0.10089989751577377, 0.013225837610661983, 0.009944065473973751, 0.06498442590236664, 0.03594978153705597, 0.00901051890105009, -0.04935517907142639, 0.028428280726075172, 0.016662482172250748, -0.07036454230546951, 0.028897425159811974, -0.012783491052687168, -0.016234586015343666, -0.029597068205475807, 0.0011979435803368688, 0.014751930721104145, 0.03274708241224289, 0.032700762152671814, -0.05388153716921806, -0.03434458374977112, 0.03882075473666191, -0.012894239276647568, -0.05786341056227684, -0.0505731925368309, 0.03648444637656212, -0.018551280722022057, -0.01095627062022686, -0.02363397181034088, 0.08503755182027817, -0.09827034920454025, 0.031581610441207886, 0.04195931553840637, -0.021482951939105988, -0.042930107563734055, 0.053916092962026596, -0.05952071771025658, 0.01013813354074955, -0.032480884343385696, 0.010553351603448391, -0.02959611266851425, 0.0004697918484453112, -0.09880625456571579, -0.026160571724176407, 0.02296098694205284, -0.04312822222709656, 0.02622532658278942, -0.019712431356310844, -0.010354968719184399, 0.042277295142412186, -0.006570827215909958, -0.030795490369200706, 0.04320602864027023, -0.07308743894100189, 0.006203257944434881, 0.01119457371532917, 0.007411148864775896, 0.10606396198272705, -0.06050681322813034, 0.06793943047523499, 0.016275765374302864, 0.04428844153881073, 0.058031484484672546, -0.031718138605356216, -0.056659016758203506, -0.025036487728357315, 0.05003930255770683, 0.014551944099366665, 0.04476001486182213, 0.03421919420361519, -0.04272487387061119, 0.020178569480776787, 0.017915284261107445, 0.04712974280118942, 0.07437659054994583, -0.037066735327243805, 0.0917411744594574, ...]"
1,My brother is in the market for a high-performance video card that supports\nVESA local bus with 1-2MB RAM. Does anyone have suggestions/ideas on:\n\n - Diamond Stealth Pro Local Bus\n\n - Orchid Farenheit 1280\n\n - ATI Graphics Ultra Pro\n\n - Any other high-performance VLB card\n\n\nPlease post or email. Thank you!\n\n - Matt\n,"[0.05006030201911926, 0.026980925351381302, -0.008864834904670715, -0.03594672307372093, 0.008585928939282894, 0.014310572296380997, -0.043572429567575455, 0.05393943190574646, -0.02893824316561222, -0.03812522068619728, -0.023232141509652138, 0.030122751370072365, -0.06470972299575806, -0.034228041768074036, 0.023131970316171646, 0.056054435670375824, 0.17435193061828613, 0.028424914926290512, 0.06454820185899734, 0.0008050971664488316, 0.01627497188746929, -0.028615452349185944, -0.005918028298765421, 0.03216960281133652, 0.035599905997514725, 0.0005109071498736739, 0.024383164942264557, 0.011950208805501461, -0.02083015628159046, -0.04016748443245888, 0.007029742933809757, 0.015071472153067589, -0.01402259804308414

### 2.3 Creo una quey y hago la búsqueda

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
query = "Russian Federation"
query_emb = model.encode(query)
print(query_emb)

[-8.52377340e-03 -3.92504930e-02 -3.42000052e-02  4.32617702e-02
 -2.49907561e-03  5.95423626e-03  8.68702233e-02 -2.59020962e-02
 -6.05973997e-04 -1.71499737e-02 -7.30644912e-02 -5.86948404e-03
 -2.40230504e-02  7.72785470e-02 -8.97390172e-02 -4.86236289e-02
 -1.98789090e-02 -3.78015079e-02 -5.36042592e-03 -2.03642733e-02
 -3.16140689e-02 -9.79854167e-02  4.54782732e-02  3.91724370e-02
  5.95646948e-02  1.77145153e-02 -2.23199418e-03 -1.09468540e-02
  5.12338197e-03 -6.64867684e-02 -3.48227508e-02 -1.31535297e-02
  5.33735752e-02 -6.93239039e-03 -1.50537333e-02  1.57261379e-02
  3.70326340e-02 -5.30506037e-02  1.98961981e-03  3.04985438e-02
 -1.14550265e-02 -1.00777559e-01 -3.67123187e-02  8.75669420e-02
 -1.14588102e-03  7.48975202e-02 -2.45475583e-02  6.63253618e-03
  1.21760396e-02  1.17584644e-02  5.32353064e-04  1.13569191e-02
 -4.64758873e-02  7.52605051e-02  1.04522556e-01 -7.02960044e-02
  2.51775179e-02 -8.67980160e-03 -6.09254427e-02 -7.77300447e-02
 -6.37419149e-03 -6.61627

In [31]:
similarities = cosine_similarity(query_emb.reshape(1,-1), embeddings)

Obtengo los 5 documentos más similares a mi query

In [32]:
z = zip(newsgroups_df.raw, similarities)
sorted(z, key=lambda x: x[1], reverse=True)

[("\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n",
  array([ 0.06407066, -0.04098997,  0.1977092 , ...,  0.03166188,
          0.04120576,  0.05887027], dtype=float32))]

In [33]:
z = zip(newsgroups_df.index, similarities)
sorted(z, key=lambda x: x[1], reverse=True)

[(0,
  array([ 0.06407066, -0.04098997,  0.1977092 , ...,  0.03166188,
          0.04120576,  0.05887027], dtype=float32))]